In [6]:
import asyncio
import aiohttp
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import nest_asyncio

nest_asyncio.apply()

client_id = '********************************'# Use your own key
client_secret = '********************************' # Use your own key
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
token = auth_manager.get_access_token(as_dict=False)

async def get_audio_features(session, track_ids):
    track_ids = [tid for tid in track_ids if tid is not None]
    if not track_ids:
        return []
    url = f"https://api.spotify.com/v1/audio-features?ids={','.join(track_ids)}"
    headers = {'Authorization': f'Bearer {token}'}
    async with session.get(url, headers=headers) as response:
        if response.status == 429:
            retry_after = int(response.headers.get('Retry-After', 60))
            print(f"Rate limit exceeded, retry after {retry_after} seconds")
            await asyncio.sleep(retry_after)
            return await get_audio_features(session, track_ids)
        audio_features = await response.json()
        return audio_features['audio_features']

async def get_playlist_features(session, playlist_id):
    offset = 0
    feature_list = []
    track_ids = []
    while True:
        url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks?offset={offset}&limit=100"
        headers = {'Authorization': f'Bearer {token}'}
        async with session.get(url, headers=headers) as response:
            if response.status == 429:
                retry_after = int(response.headers.get('Retry-After', 60))
                print(f"Rate limit exceeded, retry after {retry_after} seconds")
                await asyncio.sleep(retry_after)
                continue

            playlist = await response.json()
            if 'items' not in playlist or not playlist['items']:
                print(f"No items found in playlist {playlist_id}.")
                break

            print(f"Retrieved {len(playlist['items'])} items from playlist {playlist_id}.")
            track_ids.extend([item['track']['id'] for item in playlist['items'] if item['track'] and item['track']['id']])
            offset += len(playlist['items'])

        # Fetch audio features in batches of 100
        if track_ids:
            audio_features = await get_audio_features(session, track_ids)
            for track in playlist['items']:
                if track['track']:
                    track_id = track['track']['id']
                    features = next((x for x in audio_features if x and x['id'] == track_id), {})
                    if features:
                        track_features = {
                            'id': track['track']['id'],
                            'name': track['track']['name'],
                            'artist': track['track']['artists'][0]['name'] if track['track']['artists'] else None,
                            'album': track['track']['album']['name'],
                            'release_date': track['track']['album']['release_date'],
                            'year': int(track['track']['album']['release_date'][:4]) if track['track']['album']['release_date'] else None,
                            'popularity': track['track']['popularity'],
                            **features
                        }
                        feature_list.append(track_features)
            track_ids = []  # Clear track_ids after processing

    return pd.DataFrame(feature_list) if feature_list else pd.DataFrame()

async def fetch_all_features(playlist_ids):
    async with aiohttp.ClientSession() as session:
        tasks = [get_playlist_features(session, pid) for pid in playlist_ids]
        results = await asyncio.gather(*tasks, return_exceptions=True)
        results = [result if isinstance(result, pd.DataFrame) else pd.DataFrame() for result in results]
        return pd.concat(results, ignore_index=True) if results else pd.DataFrame()

async def main():
    playlist_ids = ['76vPilW62ExhrxEmLvrFCI', '4IP94hO9T0vJFdYboybID0', '4ogtpf7rB00YTFsSEhIeRi']
    all_features = await fetch_all_features(playlist_ids)
    if not all_features.empty:
        all_features.to_csv('spotify_features.csv', index=False)
        print('The feature data has been saved to the "spotify_features.csv"。')
    else:
        print("No data was collected.")

await main()


Retrieved 100 items from playlist 4IP94hO9T0vJFdYboybID0.
Retrieved 100 items from playlist 4ogtpf7rB00YTFsSEhIeRi.
Retrieved 100 items from playlist 76vPilW62ExhrxEmLvrFCI.
Retrieved 100 items from playlist 4ogtpf7rB00YTFsSEhIeRi.
Retrieved 100 items from playlist 4IP94hO9T0vJFdYboybID0.
Retrieved 100 items from playlist 76vPilW62ExhrxEmLvrFCI.
Retrieved 100 items from playlist 4ogtpf7rB00YTFsSEhIeRi.
Retrieved 100 items from playlist 76vPilW62ExhrxEmLvrFCI.
Retrieved 100 items from playlist 4IP94hO9T0vJFdYboybID0.
Retrieved 100 items from playlist 4ogtpf7rB00YTFsSEhIeRi.
Retrieved 100 items from playlist 76vPilW62ExhrxEmLvrFCI.
Retrieved 100 items from playlist 4IP94hO9T0vJFdYboybID0.
Retrieved 100 items from playlist 76vPilW62ExhrxEmLvrFCI.
Retrieved 100 items from playlist 4IP94hO9T0vJFdYboybID0.
Retrieved 100 items from playlist 4ogtpf7rB00YTFsSEhIeRi.
Retrieved 100 items from playlist 4IP94hO9T0vJFdYboybID0.
Retrieved 100 items from playlist 76vPilW62ExhrxEmLvrFCI.
Retrieved 100 

Retrieved 100 items from playlist 4IP94hO9T0vJFdYboybID0.
Retrieved 100 items from playlist 76vPilW62ExhrxEmLvrFCI.
Retrieved 100 items from playlist 4IP94hO9T0vJFdYboybID0.
Retrieved 100 items from playlist 76vPilW62ExhrxEmLvrFCI.
Retrieved 100 items from playlist 4IP94hO9T0vJFdYboybID0.
Retrieved 100 items from playlist 76vPilW62ExhrxEmLvrFCI.
Retrieved 100 items from playlist 4IP94hO9T0vJFdYboybID0.
Retrieved 100 items from playlist 76vPilW62ExhrxEmLvrFCI.
Retrieved 100 items from playlist 4IP94hO9T0vJFdYboybID0.
Retrieved 100 items from playlist 76vPilW62ExhrxEmLvrFCI.
Retrieved 100 items from playlist 4IP94hO9T0vJFdYboybID0.
Retrieved 100 items from playlist 76vPilW62ExhrxEmLvrFCI.
Retrieved 100 items from playlist 4IP94hO9T0vJFdYboybID0.
Retrieved 100 items from playlist 76vPilW62ExhrxEmLvrFCI.
Retrieved 100 items from playlist 4IP94hO9T0vJFdYboybID0.
Retrieved 100 items from playlist 76vPilW62ExhrxEmLvrFCI.
Retrieved 100 items from playlist 4IP94hO9T0vJFdYboybID0.
Retrieved 100 